In [ ]:
import pandas as pd 
import random

In [ ]:
df = spark.sql("SELECT * FROM AOM_Lakehouse.alert_category LIMIT 1000")
display(df)

In [ ]:
def update_value(value):
    if value>=2:
        return 'High'
    elif value==2:
        return 'Medium'
    else:
        return 'Low'

In [ ]:
def add_external_values():
    df_write = pd.DataFrame(columns=['cust_id', 'alert_impact', 'alert_type','alert_attribute','alert_date',
                                 'alert_tat','Date_action_taken','alert_status','alert_assgined_to','alert_resolved_by','account_type','industry_type'])

    df = spark.read.format("delta").load("Tables/loan_master")  
    df = df.toPandas()

    df = df.drop_duplicates(subset='cust_id', keep="first")
    
    open_prob=0.7
    # Add new columns only if they don't exist
    if 'CRILC_Defaulter_List' not in df.columns:
        df['CRILC_Defaulter_List'] = ['yes' if random.random() < open_prob else 'No' for _ in range(len(df))]
    
    if 'Wilful_Defaulter_List' not in df.columns:
        df['Wilful_Defaulter_List'] = ['yes' if random.random() < open_prob else 'No' for _ in range(len(df))]
    
    df['ratings_2023']=df['ratings_2023'].replace({'AAA':7,'AA':6,'A':5,
                                                       'BBB':4,'BB':3,'B':2,
                                                       'C':1,'D':0})
    df['ratings_2022']=df['ratings_2022'].replace({'AAA':7,'AA':6,'A':5,
                                                       'BBB':4,'BB':3,'B':2,
                                                       'C':1,'D':0})
    df['External_Rating_Alert']=df['ratings_2023']-df['ratings_2022']
    df['External_Rating_Alert']=df['External_Rating_Alert'].apply(update_value)


    names_values={
        'Regulatory_News_Alert':27,
        'Statutory_Dues_Delay':28,
        'Vendor_Loss_Alert':29,
        'Penalty_Alert':30,
        'Business_Expansion_Alert':31,
        'Competition_Alert':32,
        'Low_Market_Demand':33,
        'Employee_Attrittion':34,
        'Exchange_Filing_Alert':35,
        'Cross_Border_Alert':36,
        'Value_Chain_Alert':37,
        'Personal_Loans_Promoters':38,
        'CRILC_Defaulter_List':25,
        'Wilful_Defaulter_List':26,
        'External_Rating_Alert':24
                
             }

    for i in list(names_values.keys()):
        if names_values[i] not in [25,26]:
            df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
            df_high[i]=df[i].replace({'High':3,'Low':1,'Medium':2})
        else:
            df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
            df_high = df_high[df_high[i] == 'yes'][['cust_id', i, 'Renewal_Due_Date']]
            df_high[i]=3

        df_high.rename(columns = {i:'alert_impact'}, inplace = True)

        df_high['alert_type']=5
        df_high['alert_attribute']=names_values[i]
        df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
        df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
        df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'],format='mixed')+pd.to_timedelta(df_high['alert_tat'],unit='D' )
        open_prob=0.7
        df_high['alert_status']=['Open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
        df_high['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_high))]
        df_high['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_high))]
        df_high['account_type']=0
        df_high['industry_type']= 0
        df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
        df_high = df_high.drop_duplicates()
        df_write = pd.concat([df_write, df_high])
    return df_write

In [ ]:
df_write=add_external_values()
df_write=spark.createDataFrame(df_write)
df_write.write.format("delta").mode("overwrite").save("Tables/alert_file_4")